### User input

In [ ]:
decimal = '.'                                                            # decimal of the input file
sep = ';'                                                                # separator of the input file
input_dir = 'C:/Users/cace0002/AbspectroscoPY/results/df_pathcor_sw.csv' # input directory
output = 'C:/Users/cace0002/AbspectroscoPY/results/'                     # output directory
dateheadername = 'Timestamp'                                             # header of the date  
header = 0                                                               # header row number
dateparsingformat = '%Y-%m-%d %H:%M:%S'                                  # format of the date 
sample_name = 'sw'                                                       # name of the sample

nperiods = 10                                                            # number of dates to display in the absorbance spectra plot
fig_format = '.tiff'                                                   # format of the exported figure
dpi = 300                                                              # resolution of the exported figure

startwv = '700 nm'                                                       # starting wavelength for the baseline correction

### Start environment and import data

In [ ]:
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from datetime import datetime
from datetime import timedelta

df = pd.read_csv(input_dir, sep = sep, header = header, index_col = 0) 
df.index = pd.to_datetime(df.index)               # make sure time column (here index) is using time format
df

### Set a wavelength range to use for the baseline correction

In [ ]:
### Plot the absorbance spectra for different dates covering the temporal variability of the data and choose a wavelength range to correct for the baseline drift. 

In [ ]:
abspy.makeabsplot(df, output, dateparsingformat, nperiods, sample_name)

### abs_basecor

In [ ]:
### Compute the median of the attenuation data for the selected wavelength region and subtract it from the absorbance data to account for the instrumental baseline drift.

In [ ]:
def abs_basecor(df_in, 
                startwv):
    '''
    function to account for the instrumental baseline drift
    :argument df_in: dataframe in input
    :argument startwv: starting wavelength of the wavelength region to use for the baseline  correction
    :return: the baseline corrected attenuation dataframe    
    '''      
    df_in.columns = [str(col) + 'nm' for col in df_in.columns]
    df_out = df_in.copy()
    header = list(df_out)                            # list of wavelengths; 700-735.5 nm is the wavelength region chosen in this example
    start = df_out.columns.get_loc(startwv)          # get the starting and ending column position of the two wavelengths 
    end = len(header)                                
    med = df_out.iloc[:,start:end].median(axis=1)    # compute the median of the attenuation values for the columns between start and end 
    df_out =df_out.iloc[:,0:start]                   # obtain a subdataset which excludes columns from 700 nm onwards
    df_out =df_out.subtract(med, axis = 0)           # perform the baseline correction    
    return(df_out)

In [ ]:
df_bc = abs_basecor(df, startwv)
df_bc.to_csv(output + 'df_baselinecor_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataframe
df_bc